# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [198]:
!python create_tables.py

Tables Dropped if exists.
Tables All Created.


In [199]:
import os
import glob
import psycopg2
import pandas as pd
from datetime import datetime
from sql_queries import *
from prettytable import PrettyTable

In [200]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [201]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [202]:
filepath = get_files('data/')

song_files = get_files('data/song_data')
log_files = get_files('data/log_data')

In [203]:
# Concat all songs data

dfs = []

for file in song_files:
    x = pd.read_json(file, lines=True)
    dfs.append(x)


# Concat all songs data

df_songs = pd.concat(dfs)
df_songs

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0
...,...,...,...,...,...,...,...,...,...,...
0,1,ARAGB2O1187FB3A161,NaN,NaN,,Pucho & His Latin Soul Brothers,SOLEYHO12AB0188A85,Got My Mojo Workin,338.23302,0
0,1,AR62SOJ1187FB47BB5,NaN,NaN,,Chase & Status,SOGVQGJ12AB017F169,Ten Tonne,337.68444,2005
0,1,ARBEBBY1187B9B43DB,NaN,NaN,"Gainesville, FL",Tom Petty,SOFFKZS12AB017F194,A Higher Place (Album Version),236.17261,1994
0,1,ARNNKDK1187B98BBD5,45.80726,15.96760,Zagreb Croatia,Jinx,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),407.37914,0


In [204]:
# song data

# songs - songs in music database
# song_id, title, artist_id, year, duration

song_data = df_songs[['song_id', 'title', 'artist_id', 'year', 'duration']]

song_data = song_data.drop_duplicates('song_id')

song_data = song_data.set_index('song_id')

song_data = song_data.to_records()

song_data = list(song_data)

# convert numpy numeric to python native numeric. otherwise psycopg2 wont accept
song_data = [(x[0], x[1], x[2], int(x[3]), float(x[4])) for x in song_data]

In [205]:
# artist data

# artist_id, name, location, latitude, longitude

artist_data = df_songs[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

artist_data = artist_data.drop_duplicates('artist_id')

artist_data = artist_data.set_index('artist_id')

artist_data = artist_data.to_records()

artist_data = list(artist_data)

artist_data = [(x[0], x[1], x[2], float(x[3]), float(x[4])) for x in artist_data]

In [206]:
len(log_files)

30

In [207]:
# Concat all logs data

dfl = []

for file in log_files:
    x = pd.read_json(file, lines=True)
    dfl.append(x)


# Concat

df_logs = pd.concat(dfl)
df_logs

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,Motion City Soundtrack,Logged In,Stefany,F,1,White,205.26975,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,867,Fell In Love Without You (Acoustic),200,1543092353796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
354,Ticanaf,Logged In,Stefany,F,2,White,2594.87302,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,867,The Thousand Names of Lord Shiva (Part 1),200,1543092558796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
355,The Far East Movement,Logged In,Ryan,M,0,Smith,228.20526,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,899,Dance Like Michael Jackson,200,1543097750796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
356,the bird and the bee,Logged In,Ryan,M,1,Smith,189.51791,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,899,You're A Cad,200,1543097978796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [208]:
# df logs data

df_logs = df_logs[df_logs['page'] == 'NextSong']

df_logs['datetime'] = df_logs['ts'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))

# start_time, hour, day, week, month, year, weekday

df_time = df_logs[['ts', 'datetime']]

df_time['hour'] = df_time['datetime'].apply(lambda x: x.hour)
df_time['day'] = df_time['datetime'].apply(lambda x: x.day)
df_time['week'] = df_time['datetime'].apply(lambda x: x.week)
df_time['month'] = df_time['datetime'].apply(lambda x: x.month)
df_time['year'] = df_time['datetime'].apply(lambda x: x.year)
df_time['weekday'] = df_time['datetime'].apply(lambda x: x.weekday())


# time data and convert to python native numeric format.

time_data = df_time[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']].values
time_data = [(int(x[0]), int(x[1]), int(x[2]), int(x[3]), int(x[4]), int(x[5]), int(x[6])) for x in time_data]

print(len(time_data))

6820


/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_20076/3820926554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_logs['datetime'] = df_logs['ts'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))
/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_20076/3820926554.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['hour'] = df_time['datetime'].apply(lambda x: x.hour)
/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_20076/3820926554.py:12: SettingWithCopyWarnin

In [209]:
# load user table

# user_id, first_name, last_name, gender, level

user_df = df_logs[['userId', 'firstName', 'lastName', 'gender', 'level']]

user_data = user_df.drop_duplicates('userId', keep='last')

user_data = user_data.set_index('userId')

user_data = user_data.to_records()

user_data = list(user_data)

print(len(user_data))

# user_data = [(x[0], x[1], x[2], x[3], x[4]) for x in user_data]

# user_data


129


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [210]:
song_data[0]

('SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [211]:
# cur.execute(song_table_insert, (song_data[0]))
# conn.commit()

In [212]:
# Insert all songs data records

for row in song_data:
    # print(row)
    cur.execute(song_table_insert, row)
    conn.commit()

print('Insert song rows done')

Insert song rows done


In [213]:
cur.execute("select * from songs")
conn.commit()

t = PrettyTable(['song_id', 'title', 'artist_id', 'year', 'duration'])

for (song_id, title, artist_id, year, duration) in cur:
    t.add_row([song_id, title, artist_id, year, duration])

print(t)


+--------------------+------------------------------------------------------+--------------------+------+-----------+
|      song_id       |                        title                         |     artist_id      | year |  duration |
+--------------------+------------------------------------------------------+--------------------+------+-----------+
| SONHOTT12A8C13493C |                   Something Girls                    | AR7G5I41187FB4CE6C | 1982 | 233.40363 |
| SOIAZJW12AB01853F1 |                      Pink World                      | AR8ZCNI1187B9A069B | 1984 | 269.81832 |
| SOFSOCN12A8C143F5D |                    Face the Ashes                    | ARXR32B1187FB57099 | 2007 | 209.60608 |
| SOHKNRJ12A6701D1F8 |                     Drop of Rain                     | AR10USD1187B99F3F1 |  0   | 189.57016 |
| SOQHXMF12AB0182363 |                   Young Boy Blues                    | ARGSJW91187B9B1D6B |  0   | 218.77506 |
| SOCIWDW12A8C13D406 |                      Soul Deep   

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [214]:
# artist_data =
# artist_data

artist_data[3]


('AR10USD1187B99F3F1',
 'Tweeterfriendly Music',
 'Burlington, Ontario, Canada',
 nan,
 nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [215]:
# cur.execute(artist_table_insert, artist_data[4])
# conn.commit()

In [216]:
# Insert all artist data records

for row in artist_data:
    # print(row)
    cur.execute(artist_table_insert, row)
    conn.commit()

print('Insert artist rows done')

Insert artist rows done


In [217]:
cur.execute("select * from artists")
conn.commit()

t = PrettyTable(['artist_id', 'name', 'location', 'latitude', 'longitude'])

for (artist_id, name, location, latitude, longitude) in cur:
    t.add_row([artist_id, name, location, latitude, longitude])

print(t)

+--------------------+------------------------------------------------------------------------------------------------+-------------------------------+----------+--------------------+
|     artist_id      |                                              name                                              |            location           | latitude |     longitude      |
+--------------------+------------------------------------------------------------------------------------------------+-------------------------------+----------+--------------------+
| AR7G5I41187FB4CE6C |                                            Adam Ant                                            |        London, England        |   nan    |        nan         |
| AR8ZCNI1187B9A069B |                                        Planet P Project                                        |                               |   nan    |        nan         |
| ARXR32B1187FB57099 |                                              Gob         

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [218]:
# for i, row in time_df.iterrows():
#     cur.execute(time_table_insert, list(row))
#     conn.commit()

In [219]:
for row in time_data:
    # print(row)
    cur.execute(time_table_insert, row)
    conn.commit()

print('Insert time rows done.')

Insert time rows done.


Run `test.ipynb` to see if you've successfully added records to this table.

In [220]:
cur.execute("select * from time")
conn.commit()

t = PrettyTable(['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])

for (start_time, hour, day, week, month, year, weekday) in cur:
    t.add_row([start_time, hour, day, week, month, year, weekday])

print(t)

# start_time, hour, day, week, month, year, weekday

+---------------+------+-----+------+-------+------+---------+
|   start_time  | hour | day | week | month | year | weekday |
+---------------+------+-----+------+-------+------+---------+
| 1541903636796 |  2   |  11 |  45  |   11  | 2018 |    6    |
| 1541903770796 |  2   |  11 |  45  |   11  | 2018 |    6    |
| 1541904034796 |  2   |  11 |  45  |   11  | 2018 |    6    |
| 1541910973796 |  4   |  11 |  45  |   11  | 2018 |    6    |
| 1541911006796 |  4   |  11 |  45  |   11  | 2018 |    6    |
| 1541930188796 |  9   |  11 |  45  |   11  | 2018 |    6    |
| 1541930447796 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 1541930647796 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 1541930908796 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 1541931120796 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 1541932063796 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 1541932424796 |  10  |  11 |  45  |   11  | 2018 |    6    |
| 1541932582796 |  10  |  11 |  45  |   11  | 2018 |   

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [221]:
for row in user_data:
    # print(row)
    cur.execute(user_table_insert, row)
    conn.commit()

print('Insert users done.')


Insert users done.


Run `test.ipynb` to see if you've successfully added records to this table.

In [222]:
cur.execute("select * from users")
conn.commit()

t = PrettyTable(['user_id', 'first_name', 'last_name', 'gender', 'level'])

for (user_id, first_name, last_name, gender, level) in cur:
    t.add_row([user_id, first_name, last_name, gender, level])

print(t)

# user_id, first_name, last_name, gender, level

+---------+------------+-----------+--------+-------+
| user_id | first_name | last_name | gender | level |
+---------+------------+-----------+--------+-------+
|    56   |   Cienna   |  Freeman  |   F    |  free |
|    59   |    Lily    |   Cooper  |   F    |  free |
|    20   |   Aiden    |  Ramirez  |   M    |  paid |
|    72   |   Hayden   |   Brock   |   F    |  paid |
|    48   |   Marina   |   Sutton  |   F    |  free |
|   101   |   Jayden   |    Fox    |   M    |  free |
|    83   |  Stefany   |   White   |   F    |  free |
|    48   |   Marina   |   Sutton  |   F    |  free |
|    17   |  Makinley  |   Jones   |   F    |  free |
|    89   |  Kynnedi   |  Sanchez  |   F    |  free |
|    49   |   Chloe    |   Cuevas  |   F    |  free |
|   100   |   Adler    |  Barrera  |   M    |  free |
|    15   |    Lily    |    Koch   |   F    |  paid |
|    80   |   Tegan    |   Levine  |   F    |  free |
|    86   |   Aiden    |    Hess   |   M    |  free |
|    88   |  Mohammad  | Rod

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [223]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

df = df_logs.copy()

In [224]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent


for index, row in df.iterrows():

    # songplay_id = None auro increment by db
    start_time = row.ts
    user_id = row.userId
    level = row.level
    song_id = None
    song_title = row.song
    artist_id = None
    artist_name = row.artist
    session_id = row.sessionId
    location = row.location
    user_agent = row.userAgent

    params = (song_title, artist_name)
    print(params)


    # get songid and artistid from song and artist tables
    cur.execute(song_select, params)
    results = cur.fetchone()
    # print(f"results: {results}")

    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay record
    songplay_data = (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

print("Insert songplay records done")

('Fuck Kitty', 'Frumpies')
('By The Time This Night Is Over', 'Kenny G with Peabo Bryson')
('God & Satan', 'Biffy Clyro')
('Beautiful', 'HIM')
('Supreme Balloon', 'Matmos')
('The One', 'Gary Allan')
('Five Roses', 'Miracle Fortress')
('Cuentale', 'Don Omar')
("D'Evils", 'Jay-Z')
('Easily (Album Version)', 'Red Hot Chili Peppers')
('Rebels of the Sacred Heart', 'Flogging Molly')
('Now_ Right Now', 'Reverend Horton Heat')
('I Made A Resolution', 'Sea Wolf')
('Lucky (Album Version)', 'Jason Mraz & Colbie Caillat')
('Enough\x19s Enough', 'Jamie Lidell')
('Mushaboom (Postal Service Mix)', 'Feist')
("We're Going Out Tonight", 'Sex Slaves')
('To Them These Streets Belong', 'Rise Against')
('Get Me Bodied', 'BeyoncÃ\x83Â©')
('Never Leave Me Alone', 'Nate Dogg')
('Love Story', 'Taylor Swift')
('Sweet home Alabama', 'Lynyrd Skynyrd')
('Caught Out There (Explicit)', 'Kelis')
('Last Day Of Magic', 'The Kills')
('What A Feeling', 'Collie Buddz featuring Paul Wall')
('Fireflies', 'Charttraxx Karaoke

Run `test.ipynb` to see if you've successfully added records to this table.

In [225]:
cur.execute("select * from songplays")
conn.commit()

t = PrettyTable(['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent'])

for (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) in cur:
    t.add_row([songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent])

print(t)

# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

+-------------+---------------+---------+-------+--------------------+--------------------+------------+------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| songplay_id |   start_time  | user_id | level |      song_id       |     artist_id      | session_id |                    location                    |                                                                  user_agent                                                                 |
+-------------+---------------+---------+-------+--------------------+--------------------+------------+------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|      1      | 1541903636796 |    69   |  free |        None        |        None        |    455     |  Philad

# Close Connection to Sparkify Database

In [226]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [229]:
# test_data = ('Love Story', 'Taylor Swift')
#
# cur.execute("""
# select song_id, artist_id
# from songs
# inner join artists
# using (artist_id)
# where title = %s and name = %s
# """, test_data)
# conn.commit()
#
# res = cur.fetchone()
# print(res)
#
# # res = cur.fetchone()
#
# # t = PrettyTable(['song_id', 'artist_id'])
# # for (a, b) in cur:
# #     t.add_row([a, b])
# #
# # print(t)

None


In [233]:
df_songs['title'].value_counts()

Something Girls                                        1
OAKtown                                                1
Native Soul                                            1
Der Kleine Dompfaff                                    1
Streets On Fire (Explicit Album Version)               1
                                                      ..
The Emperor Falls                                      1
Setanta matins                                         1
¿Dónde va Chichi?                                      1
I Think My Wife Is Running Around On Me (Taco Hell)    1
Clementina Santafè                                     1
Name: title, Length: 71, dtype: int64

In [235]:
df_logs['song'].value_counts()

You're The One                                                         37
Undo                                                                   28
Revelry                                                                27
Sehr kosmisch                                                          21
Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)    19
                                                                       ..
Better Is One Day (Stereo Trax)                                         1
Two Shoes                                                               1
Next Evil                                                               1
Fighters (feat. Matthew Santos) (Amended Album Version)                 1
Inevitable                                                              1
Name: song, Length: 5189, dtype: int64

In [254]:
test_data = song_data[0]



test_data = list(test_data)

for x in test_data:
    print(type(x))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'float'>
